In [1]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from confluent_kafka import Consumer, KafkaException
import boto3
import json

# Change the IP and port as needed
bootstrap_servers = ':9092'

# Consumer configuration
consumer_config = {
    'bootstrap.servers': bootstrap_servers,
    'group.id': 'my_consumer_group',
    'auto.offset.reset': 'earliest',  # Change to 'latest' if you want to start consuming from the latest offset
    'enable.auto.commit': True,
    'auto.commit.interval.ms': 1000,
}

# Create a Kafka consumer instance
consumer = Consumer(consumer_config)

# Subscribe to the 'demo_test' topic
consumer.subscribe(['demo_test'])

# S3 Configuration
s3_bucket_name = 'your_bucket_name'
s3_prefix = ''

# Create an S3 client
s3 = boto3.client('s3')

try:
    while True:
        # Poll for messages
        message = consumer.poll(1.0)  # Adjust the timeout as needed

        if message is not None:
            # Decode and print the message value
            message_value = message.value().decode('utf-8')
            print(f"Received message: {message_value}")

            # Upload the message to S3
            s3_key = f"{s3_prefix}{message.offset()}.json"
            s3.put_object(Body=message_value, Bucket=s3_bucket_name, Key=s3_key)

except KeyboardInterrupt:
    pass
finally:
    # Close down consumer to commit final offsets.
    consumer.close()
